# Sensitivity Analysis of Kv4.3 Model

## Luca Sagresti

## 29/01/2020

## Libraries Import

In [1]:
%clear
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy.integrate import odeint
from scipy.optimize import curve_fit
from scipy.optimize import minimize
from scipy.optimize import least_squares
from SALib.sample import saltelli
from SALib.analyze import sobol


%matplotlib inline

mpl.rcParams['figure.dpi']=100
mpl.rcParams['figure.titlesize']=20
mpl.rcParams['axes.facecolor']='white'        
mpl.rcParams['lines.linewidth']=2.0
mpl.rcParams['axes.linewidth']=2.0
mpl.rcParams['xtick.major.pad']=6
mpl.rcParams['ytick.major.pad']=6
mpl.rcParams['xtick.labelsize']=14
mpl.rcParams['ytick.labelsize']=14
mpl.rcParams['axes.titlesize']=18
mpl.rcParams['axes.labelsize']=18
mpl.rcParams['axes.grid']='True'
mpl.rcParams['axes.axisbelow']='line'
mpl.rcParams['legend.fontsize']=12

## Import Experimental Data

In [2]:
import pandas as pd

folder="~/Notebook/Data_exp_Kv/"

# Activation dataset

dataset_act = pd.read_csv(folder+"Act_datasets.csv", skiprows=1)

act_data = dataset_act.to_numpy()

act_data_WT = act_data[:,1]

act_data_S390N = act_data[:,3]

# Inactivation dataset

dataset_inact = pd.read_csv(folder+"Inact_datasets.csv", skiprows=1)

inact_data = dataset_inact.to_numpy()

inact_data_WT = inact_data[:,1]

inact_data_S390N = inact_data[:,3]

# Recovery dataset

dataset_rec = pd.read_csv(folder+"Rec_datasets.csv", skiprows=1)

rec_data = dataset_rec.to_numpy()

rec_data_WT = rec_data[:,1]

rec_data_S390N = rec_data[:,3]

## Activation

In [3]:
def ode_act_WT (S, t, p):
    
    C0=S[0]
    C1=S[1]
    C2=S[2]
    C3=S[3]
    C4=S[4]
    I0=S[5]
    I1=S[6]
    I2=S[7]
    I3=S[8]
    I4=S[9]
    O =S[10]

    
    #constants
    
    T = 291.0 #K or 18 degree celsius
    e =  1.602176634 * (10**-19.0) # C
    K_B = 1.380649 * (10**-23.0) # J*K^-1
    
    exp_factor = (e/(K_B * T)) * (10**-3) 

    #Voltage sequences
    
    V = 0.0 #mV 

    if 0 <= t < 0.5:
        V = -90.0 # mV
        
    if 0.5 <= t < 0.55:
        V = p[11] # Vtest
        
    if 0.55 <= t <= 1.05:
        V = -50.0 # mV
        
    if t > 1.05 :
        V = -50.0 #mV
    
    
    #voltage dependent rate constants
     
    alpha = p[0] * np.exp(p[1] * (V * exp_factor))
    beta = p[2] * np.exp(-1.0 * p[3] * (V * exp_factor))
    k_CO = p[4] * np.exp(p[5] * (V * exp_factor))
    k_OC = p[6] * np.exp(-1.0 * p[7] * (V * exp_factor))
    
    
    k_CI = p[8]
    
    k_IC = p[9]

    f = p[10]
    
    # ODEs
    
    dC0dt = beta * C1 + (k_IC/(f**4.0)) * I0 - (k_CI*(f**4.0) + 4.0 * alpha) * C0
    dC1dt = 4.0 * alpha * C0 + 2.0 * beta * C2 + (k_IC/(f**3.0)) * I1 - (k_CI*(f**3.0) + beta + 3.0 * alpha) * C1 
    dC2dt = 3.0 * alpha * C1 + 3.0 * beta * C3 + (k_IC/(f**2.0)) * I2 - (k_CI*(f**2.0) + 2.0 * beta + 2.0 * alpha) * C2 
    dC3dt = 2.0 * alpha * C2 + 4.0 * beta * C4 + (k_IC/f) * I3 - (k_CI*f + 3.0 * beta + 1.0 * alpha) * C3 
    dC4dt = 1.0 * alpha * C3 + k_OC * O + k_IC * I4 - (k_CI + k_CO + 4.0 * beta) * C4 
    
    dI0dt = beta * f * I1 + (k_CI*(f**4.0)) * C0 - (k_IC/(f**4.0) + 4.0 * (alpha/f)) * I0
    dI1dt = 4.0 * (alpha/f) * I0 + 2.0 * beta * f * I2 + (k_CI*(f**3.0)) * C1 - (k_IC/(f**3.0) + beta * f + 3.0 * (alpha/f)) * I1 
    dI2dt = 3.0 * (alpha/f) * I1 + 3.0 * beta * f * I3 + (k_CI*(f**2.0)) * C2 - (k_IC/(f**2.0) + 2.0 * beta * f + 2.0 * (alpha/f)) * I2 
    dI3dt = 2.0 * (alpha/f) * I2 + 4.0 * beta * f * I4 + (k_CI*f) * C3 - (k_IC/f + 3.0 * beta * f + 1.0 * (alpha/f)) * I3 
    dI4dt = 1.0 * (alpha/f) * I3 + k_CI * C4 - (k_IC + 4.0 * beta * f) * I4     
    
    dOdt = k_CO * C4 - k_OC * O
    
    return (dC0dt, dC1dt, dC2dt, dC3dt, dC4dt, dI0dt, dI1dt, dI2dt, dI3dt, dI4dt, dOdt)

In [4]:
def Act_Protocol(max_V, DeltaV):
    Vhold = -90.0 #mV
    
    Vtest = np.linspace(Vhold,max_V,np.abs(np.int((max_V-Vhold)/DeltaV))+1)
    
    return Vtest #array of testing voltages

In [5]:
def Boltz_eq(x, V1_2, k):
    return 1.0/(1.0 + np.exp(-(x-V1_2)/k))

In [6]:
def Act_LS_func(x, teta):

    # conductance parameters

    #EK      = 0.0    # mV
    gK_max  = 33.2     # nS 

    # Membrane capacitance
    Cm      = 1.0    # microF cm^-2
    
    #define activation sequence protocol
    
    Vmax = 60.0 #mV
    
    increment = 10.0 #mV
    
    Vtest = Act_Protocol(Vmax, increment)
    
    # Time discretiztion
    
    tend = 1.05
    
    ttest_i = 0.50 #time at which you start record the current
    
    ttest_f = 0.55 #time at which you end record the current
     
    Npoints = 200000
    
    Points_per_sec = np.int(Npoints/tend) 
    

    # prepare empty arrays
    Open_states = np.zeros((Npoints,len(Vtest)))

    max_conductance = np.zeros(len(Vtest)) 

    
    for i in range (0,len(Vtest)):
        gamma = np.append(teta, Vtest[i])
        
        f = lambda S,t: ode_act_WT(S, t, gamma)
        
        r = odeint(f, S0, x)
        
        Open_states[:,i] = r[:,10]
        
        max_conductance[i] = gK_max * np.amax(r[np.int(Points_per_sec*ttest_i):np.int(Points_per_sec*ttest_f),10])
    
    
    g_gmax = (max_conductance)/(np.amax(max_conductance))
    
    return g_gmax

## Inactivation

In [7]:
def ode_inac_WT(C, t, p):
    C0=C[0]
    C1=C[1]
    C2=C[2]
    C3=C[3]
    C4=C[4]
    I0=C[5]
    I1=C[6]
    I2=C[7]
    I3=C[8]
    I4=C[9]
    O=C[10]

    
    #constants
    
    T = 291.0 #K or 18 degree celsius
    e =  1.602176634 * (10**-19.0) # C
    K_B = 1.380649 * (10**-23.0) # J*K^-1
    
    exp_factor = (e/(K_B * T)) * (10**-3) 

    #Voltage sequences
    
    V = 0.0 #mV 

    if 0 <= t < 1.00:
        V=-90.0
        
    if 1.00 <= t < 2.00:
        V=p[11]
        
    if 2.00 <= t <= 3.00:
        V=60.0
        
    if t > 3.00 :
        V=60.0
 
    # wild type parameters

    k_CI = p[8]
    
    k_IC = p[9]

    f = p[10]
    
    #voltage dependent rate constants
     
    alpha = p[0] * np.exp(p[1] * (V * exp_factor))
    beta = p[2] * np.exp(-1.0 * p[3] * (V * exp_factor))
    k_CO = p[4] * np.exp(p[5] * (V * exp_factor))
    k_OC = p[6] * np.exp(-1.0 * p[7] * (V * exp_factor))
    
    # ODEs
    
    dC0dt = beta * C1 + (k_IC/(f**4.0)) * I0 - (k_CI*(f**4.0) + 4.0 * alpha) * C0
    dC1dt = 4.0 * alpha * C0 + 2.0 * beta * C2 + (k_IC/(f**3.0)) * I1 - (k_CI*(f**3.0) + beta + 3.0 * alpha) * C1 
    dC2dt = 3.0 * alpha * C1 + 3.0 * beta * C3 + (k_IC/(f**2.0)) * I2 - (k_CI*(f**2.0) + 2.0 * beta + 2.0 * alpha) * C2 
    dC3dt = 2.0 * alpha * C2 + 4.0 * beta * C4 + (k_IC/f) * I3 - (k_CI*f + 3.0 * beta + 1.0 * alpha) * C3 
    dC4dt = 1.0 * alpha * C3 + k_OC * O + k_IC * I4 - (k_CI + k_CO + 4.0 * beta) * C4 
    
    dI0dt = beta * f * I1 + (k_CI*(f**4.0)) * C0 - (k_IC/(f**4.0) + 4.0 * (alpha/f)) * I0
    dI1dt = 4.0 * (alpha/f) * I0 + 2.0 * beta * f * I2 + (k_CI*(f**3.0)) * C1 - (k_IC/(f**3.0) + beta * f + 3.0 * (alpha/f)) * I1 
    dI2dt = 3.0 * (alpha/f) * I1 + 3.0 * beta * f * I3 + (k_CI*(f**2.0)) * C2 - (k_IC/(f**2.0) + 2.0 * beta * f + 2.0 * (alpha/f)) * I2 
    dI3dt = 2.0 * (alpha/f) * I2 + 4.0 * beta * f * I4 + (k_CI*f) * C3 - (k_IC/f + 3.0 * beta * f + 1.0 * (alpha/f)) * I3 
    dI4dt = 1.0 * (alpha/f) * I3 + k_CI * C4 - (k_IC + 4.0 * beta * f) * I4     
    
    dOdt = k_CO * C4 - k_OC * O
    
    return (dC0dt, dC1dt, dC2dt, dC3dt, dC4dt, dI0dt, dI1dt, dI2dt, dI3dt, dI4dt, dOdt)

In [8]:
def Inact_Protocol(max_V, DeltaV):
    Vhold = -90.0 #mV
    
    Vtest = np.linspace(Vhold,max_V,np.abs(np.int((max_V-Vhold)/DeltaV))+1)
    
    return Vtest #array of testing voltages

In [9]:
def Boltz_eq_in(x, V1_2, k):
    return 1.0/(1.0 + np.exp((x-V1_2)/k))

In [10]:
def Inact_LS_func(x, teta):

    # conductance parameters

    EK      = -90.0    # mV
    gK_max  = 33.2     # nS 

    # Membrane capacitance
    Cm      = 1.0    # microF cm^-2

    
    Vtesting =  60.0 #mV
    
    
    #define activation sequence protocol
    
    Vmax = 60.0 #mV
    
    increment = 10.0 #mV
    
    Vtest = Inact_Protocol(Vmax, increment)
    
    # Time discretiztion
    
    tend = 3.00
    
    ttest = 2.00 #time at which you start record the current
     
    Npoints = 200000
    
    Points_per_sec = np.int(Npoints/tend) 
    

    # prepare empty arrays
    Open_states = np.zeros((Npoints,len(Vtest)))

    max_conductance = np.zeros(len(Vtest)) 

    max_currents = np.zeros(len(Vtest)) 
    
    for i in range (0,len(Vtest)):
        gamma = np.append(teta, Vtest[i])
        
        f = lambda S,t: ode_inac_WT(S, t, gamma)
        
        r = odeint(f, S0, x)
        
        Open_states[:,i] = r[:,10]
        
        max_conductance[i] = gK_max * np.amax(r[np.int(Points_per_sec*ttest):,10]-r[np.int(Points_per_sec*ttest)-1,10])
        
        max_currents[i] = max_conductance[i] * (Vtesting - EK)
    
    I_Imax = (max_currents)/(np.amax(max_currents))
    
    return I_Imax

## Recovery from Inactivation

In [11]:
def ode_rec_WT (C, t, p):
    
    C0=C[0]
    C1=C[1]
    C2=C[2]
    C3=C[3]
    C4=C[4]
    I0=C[5]
    I1=C[6]
    I2=C[7]
    I3=C[8]
    I4=C[9]
    O=C[10]

    
    #constants
    
    T = 291.0 #K or 18 degree celsius
    e =  1.602176634 * (10**-19.0) # C
    K_B = 1.380649 * (10**-23.0) # J*K^-1
    
    exp_factor = (e/(K_B * T)) * (10**-3) 

    #Voltage sequences
    
    V = 0.0 #mV 

    if 0 <= t < 1.00:
        V=-90.0
        
    if 1.00 <= t < 2.00:
        V=60.0
        
    if 2.00 <= t <= (2.00 + p[11]):
        V=-90.0
        
    if (2.00 + p[11]) < t <= (p[11] + 3.00):
        V=60.0
 
    if t > (p[11] + 3.00):
        V=-90.0

    # wild type parameters

    k_CI = p[8]
    
    k_IC = p[9]

    f = p[10]
    
    #voltage dependent rate constants
       
    alpha = p[0] * np.exp(p[1] * (V * exp_factor))
    beta = p[2] * np.exp(-1.0 * p[3] * (V * exp_factor))
    k_CO = p[4] * np.exp(p[5] * (V * exp_factor))
    k_OC = p[6] * np.exp(-1.0 * p[7] * (V * exp_factor))
    
    # ODEs
    
    dC0dt = beta * C1 + (k_IC/(f**4.0)) * I0 - (k_CI*(f**4.0) + 4.0 * alpha) * C0
    dC1dt = 4.0 * alpha * C0 + 2.0 * beta * C2 + (k_IC/(f**3.0)) * I1 - (k_CI*(f**3.0) + beta + 3.0 * alpha) * C1 
    dC2dt = 3.0 * alpha * C1 + 3.0 * beta * C3 + (k_IC/(f**2.0)) * I2 - (k_CI*(f**2.0) + 2.0 * beta + 2.0 * alpha) * C2 
    dC3dt = 2.0 * alpha * C2 + 4.0 * beta * C4 + (k_IC/f) * I3 - (k_CI*f + 3.0 * beta + 1.0 * alpha) * C3 
    dC4dt = 1.0 * alpha * C3 + k_OC * O + k_IC * I4 - (k_CI + k_CO + 4.0 * beta) * C4 
    
    dI0dt = beta * f * I1 + (k_CI*(f**4.0)) * C0 - (k_IC/(f**4.0) + 4.0 * (alpha/f)) * I0
    dI1dt = 4.0 * (alpha/f) * I0 + 2.0 * beta * f * I2 + (k_CI*(f**3.0)) * C1 - (k_IC/(f**3.0) + beta * f + 3.0 * (alpha/f)) * I1 
    dI2dt = 3.0 * (alpha/f) * I1 + 3.0 * beta * f * I3 + (k_CI*(f**2.0)) * C2 - (k_IC/(f**2.0) + 2.0 * beta * f + 2.0 * (alpha/f)) * I2 
    dI3dt = 2.0 * (alpha/f) * I2 + 4.0 * beta * f * I4 + (k_CI*f) * C3 - (k_IC/f + 3.0 * beta * f + 1.0 * (alpha/f)) * I3 
    dI4dt = 1.0 * (alpha/f) * I3 + k_CI * C4 - (k_IC + 4.0 * beta * f) * I4     
    
    dOdt = k_CO * C4 - k_OC * O
    
    return (dC0dt, dC1dt, dC2dt, dC3dt, dC4dt, dI0dt, dI1dt, dI2dt, dI3dt, dI4dt, dOdt)

In [12]:
def Rec_Protocol(max_t, Deltat):
    min_t = 0.015 #mV
    
    t_pulse = np.linspace(min_t,max_t,np.abs(np.int((max_t-min_t)/Deltat))+1)
    
    return t_pulse #array of testing prepulses

In [13]:
def Simple_exp_eq(x,tau):
    return 1.0 - np.exp(-(x - 15.0 )/ tau)

In [14]:
def Rec_LS_func(x, teta):

    # conductance parameters

    EK      = -90.0    # mV
    gK_max  = 33.2      # nS 

    # Membrane capacitance
    Cm      = 1.0    # microF cm^-2
    
    #define activation sequence protocol
    
    tmax = 0.300 #s
    
    increment = 0.015 #s
    
    t_pulse = Rec_Protocol(tmax, increment)
    
    tini_prep = 1.00 #s
    
    tend_prep = 2.00 #s
    
    Vtesting = 60.0 #mV    

    # Time discretiztion
    
    tend = 4.00 #s
     
    Npoints = 200000
    
    Points_per_sec = np.int(Npoints/tend) 
    
    # prepare empty arrays
    Open_states = np.zeros((Npoints,len(t_pulse)))

    max_conductance = np.zeros(len(t_pulse)) 
    
    max_currents = np.zeros(len(t_pulse))

    max_conductance_prep = np.zeros(len(t_pulse))

    max_currents_prep = np.zeros(len(t_pulse))
    
    for i in range (0,len(t_pulse)):
        gamma = np.append(teta, t_pulse[i])
        
        f = lambda S,t: ode_rec_WT(S, t, gamma)
        
        r = odeint(f, S0, x)
        
        Open_states[:,i] = r[:,10]
        
        max_conductance[i] = gK_max * np.amax(r[np.int(Points_per_sec*(tend_prep+t_pulse[i])):,10])
        
        max_conductance_prep[i] = gK_max * np.amax(r[np.int(Points_per_sec*tini_prep):np.int(Points_per_sec*tend_prep),10])
    
        # Compute the current proportional to the open channel conductance and potential applied
    
        max_currents[i] = max_conductance[i] * (Vtesting - EK)
    
        max_currents_prep[i] = max_conductance_prep[i] * (Vtesting - EK) # nS * mV = pA
    
    I_Imax = np.true_divide(max_currents,max_currents_prep) 

    return I_Imax

## Cost Function

In [15]:
def residual_Tot(p):

    # Time discretiztion
    tini = 0.0
    
    tend = 4.00
    
#    ttest = 0.56 #time at which you start record the current
     
    Npoints = 200000
    
    Points_per_sec = np.int(Npoints/tend) 
    
    # time array
    t = np.linspace(tini,tend,Npoints)

####################### START OF THE 3 VOLTAGE PROTOCOLS ###################   

    # DEFINE ACTIVATION SEQUENCE PROTOCOL
    
    Vmax = 60.0 #mV
    
    increment = 10.0 #mV
    
    Vtest_act = Act_Protocol(Vmax, increment)
    
    #experimental data for hemiactivation and slope
    
    exp_Vhemi_act = -22.70 #mV
    
    exp_k_act = 13.00 #mV
    
    #evaluation of experimental points trough the declared Boltzmann fit
    
    exp_data_act = Boltz_eq(Vtest_act,exp_Vhemi_act,exp_k_act)
    
    # here data from perfect fit
    #sq_err_act = np.sum(np.subtract(exp_data_act,Act_LS_func(t,p))**2)
    
    #here experimental data imported with Pandas in initial section
    sq_err_act = np.sum(np.subtract(act_data_WT,Act_LS_func(t,p))**2)
    
    act_cost_func = (1.0/len(Vtest_act)) * sq_err_act
    
    
    # DEFINE INACTIVATION SEQUENCE PROTOCOL
    
#    Vmax = 60.0 #mV
    
#    increment = 10.0 #mV
    
    Vtest_inact = Inact_Protocol(Vmax, increment)
    
    #experimental data for hemiactivation and slope
    
    exp_Vhemi_inact = -49.60 #mV
    
    exp_k_inact = 5.10 #mV
    
    #evaluation of experimental points trough the declared Boltzmann fit
    
    exp_data_inact = Boltz_eq_in(Vtest_inact,exp_Vhemi_inact,exp_k_inact)
    
    # here data from perfect fit
    #sq_err_inact = np.sum(np.subtract(exp_data_inact,Inact_LS_func(t,p))**2)
    
    #here experimental data imported with Pandas in initial section
    sq_err_inact = np.sum(np.subtract(inact_data_WT,Inact_LS_func(t,p))**2)
    
    inact_cost_func = (1.0/len(Vtest_inact)) * sq_err_inact
   
   
    # DEFINE RECOVERY SEQUENCE PROTOCOL
    
    tmax = 300.0 #ms
    
    increment = 15.0 #ms
    
    t_pulse = Rec_Protocol(tmax, increment)
    
    #experimental data for recovery time
    
    exp_tau = 47.0 #ms
    
    #evaluation of experimental points trough the declared exponential fit
    
    exp_data_rec = Simple_exp_eq(t_pulse,exp_tau)
    
    # here data from perfect fit
    #sq_err_rec = np.sum(np.subtract(exp_data_rec,Rec_LS_func(t,p))**2)
    
    #here experimental data imported with Pandas in initial section
    sq_err_rec = np.sum(np.subtract(rec_data_WT,Rec_LS_func(t,p))**2)
    
    rec_cost_func = (1.0/len(t_pulse)) * sq_err_rec
    
    
    # sum of the three protocols cost function
    

    return (act_cost_func + inact_cost_func + rec_cost_func)

## Sensitivity Analysis

In [16]:
C0_0 = 439.0
C1_0 = 258.8
C2_0 = 57.2
C3_0 = 5.6
C4_0 = 0.2
I0_0 = 12.8
I1_0 = 55.3
I2_0 = 89.4
I3_0 = 64.2
I4_0 = 17.2
O_0  = 0.1

# Pack up the initial conditions:

S0 = [C0_0, C1_0, C2_0, C3_0, C4_0, I0_0, I1_0, I2_0, I3_0, I4_0, O_0]


problem ={
    'num_vars' : 11,
    'names' : ['alpha_0','alpha_1','beta_0','beta_1','KCO_0','KCO_1','KOC_0','KOC_1','KCI','KIC','f'],
    'bounds' : [[100.0,10000.0],
                [0.0,1.0],
                [0.0,400.0],
                [0.0,10.0],
                [200.0,1000.0],
                [0.0,1.0],
                [0.0,300.0],
                [0.0,1.0],
                [0.0,400.0],
                [0.0,200.0],
                [0.0,0.8]]
}

param_values=saltelli.sample(problem,500)

Y = np.zeros([param_values.shape[0]])

for i, X in enumerate(param_values):
    
    Y[i] = residual_Tot(X)

    
# model output vector Y
Y[np.isnan(Y)] = np.nanmean(Y)

Si = sobol.analyze(problem, Y, print_to_console=True)


/home/gbrancato/environments/my_env_p3/lib/python3.5/site-packages/scipy/integrate/odepack.py:248: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
/home/gbrancato/environments/my_env_p3/lib/python3.5/site-packages/ipykernel_launcher.py:50: RuntimeWarning: invalid value encountered in true_divide
/home/gbrancato/environments/my_env_p3/lib/python3.5/site-packages/ipykernel_launcher.py:54: RuntimeWarning: invalid value encountered in true_divide
/home/gbrancato/environments/my_env_p3/lib/python3.5/site-packages/ipykernel_launcher.py:63: RuntimeWarning: invalid value encountered in true_divide
/home/gbrancato/environments/my_env_p3/lib/python3.5/site-packages/scipy/integrate/odepack.py:248: ODEintWarning: Repeated error test failures (internal error). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
/home/gbrancat

Parameter S1 S1_conf ST ST_conf
alpha_0 0.070862 0.094789 0.141723 0.049679
alpha_1 0.059051 0.092807 0.141723 0.053386
beta_0 0.059051 0.121170 0.224395 0.064447
beta_1 0.129913 0.171539 0.543273 0.100555
KCO_0 0.023621 0.090645 0.118103 0.052887
KCO_1 0.035431 0.094531 0.100387 0.045155
KOC_0 -0.023621 0.121237 0.188964 0.057954
KOC_1 0.011810 0.121647 0.200775 0.063043
KCI -0.023621 0.123067 0.129913 0.058184
KIC 0.011810 0.132436 0.212585 0.064654
f -110.283248 244.336812 519550.347364 1146051.431448

Parameter_1 Parameter_2 S2 S2_conf
alpha_0 alpha_1 -0.094482 0.137428
alpha_0 beta_0 -0.094482 0.137428
alpha_0 beta_1 -0.094482 0.132873
alpha_0 KCO_0 -0.082672 0.137253
alpha_0 KCO_1 -0.082672 0.137253
alpha_0 KOC_0 -0.070862 0.137175
alpha_0 KOC_1 -0.082672 0.138436
alpha_0 KCI -0.082672 0.137930
alpha_0 KIC -0.082672 0.138436
alpha_0 f -0.082672 0.129149
alpha_1 beta_0 -0.118103 0.136486
alpha_1 beta_1 -0.118103 0.126844
alpha_1 KCO_0 -0.106293 0.130292
alpha_1 KCO_1 -0.106293 0.1